In [1]:
from resnet import ResNet, BasicBlock, Bottleneck

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18, resnet50

from tqdm.auto import tqdm

import numpy as np
import os
import random

c:\Users\jeong\.conda\envs\papers-with-code\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def seed_everything(seed=2023):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything()

In [3]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [5]:
my_resnet18 = ResNet(BasicBlock, [2, 2, 2, 2])
my_resnet50 = ResNet(Bottleneck, [3, 4, 6, 3])

In [6]:
learning_rate = 0.1
optimizer_my_resnet18 = optim.SGD(my_resnet18.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0001)
optimizer_my_resnet50 = optim.SGD(my_resnet50.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0001)

In [7]:
def train(model, train_loader, optimizer, device):
    model.to(device)
    model.train()
    criterion = nn.CrossEntropyLoss().to(device)

    for epoch in range(20):
        train_loss = []
        train_acc = []

        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.to(device)
            labels = labels.to(device)

            # optimizer에 저장된 미분값을 0으로 초기화
            optimizer.zero_grad()

            # 모델에 이미지 forward
            pred_logit = model(imgs)

            # loss 값 계산
            loss = criterion(pred_logit, labels)

            # Backpropagation
            loss.backward()
            optimizer.step()

            # Accuracy 계산
            pred_label = torch.argmax(pred_logit, 1)
            acc = (pred_label == labels).sum().item() / len(imgs)

            train_loss.append(loss.item())
            train_acc.append(acc)

        _train_loss = np.mean(train_loss)
        _train_acc = np.mean(train_acc)

        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}], Train Accuracy : [{_train_acc:.5f}]')

In [8]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    
    with torch.no_grad():
        test_acc = []

        for imgs, labels in tqdm(iter(test_loader)):
            imgs = imgs.to(device)
            labels = labels.to(device)
            
            pred_logit = model(imgs)
            pred_label = torch.argmax(pred_logit, 1)
            acc = (pred_label == labels).sum().item() / len(imgs)
            test_acc.append(acc)
        
        _test_acc = np.mean(test_acc)
        print(f'Test Accuracy: [{_test_acc:.5f}]')

In [9]:
train(my_resnet18, train_loader, optimizer_my_resnet18, device)

100%|██████████| 391/391 [00:14<00:00, 27.72it/s]


Epoch [0], Train Loss : [2.19542], Train Accuracy : [0.28512]


100%|██████████| 391/391 [00:07<00:00, 55.82it/s]


Epoch [1], Train Loss : [1.56376], Train Accuracy : [0.42534]


100%|██████████| 391/391 [00:07<00:00, 53.30it/s]


...

Epoch [17], Train Loss : [0.62983], Train Accuracy : [0.78068]


100%|██████████| 391/391 [00:07<00:00, 51.37it/s]


Epoch [18], Train Loss : [0.61242], Train Accuracy : [0.78675]


100%|██████████| 391/391 [00:06<00:00, 57.43it/s]

Epoch [19], Train Loss : [0.59871], Train Accuracy : [0.79132]


In [10]:
inference(my_resnet18, test_loader, device)

100%|██████████| 100/100 [00:01<00:00, 75.57it/s]

Test Accuracy: [0.76680]


In [11]:
train(my_resnet50, train_loader, optimizer_my_resnet50, device)

100%|██████████| 391/391 [00:19<00:00, 19.57it/s]


Epoch [0], Train Loss : [4.32598], Train Accuracy : [0.15267]


100%|██████████| 391/391 [00:14<00:00, 26.63it/s]


Epoch [1], Train Loss : [2.05697], Train Accuracy : [0.21513]


100%|██████████| 391/391 [00:14<00:00, 26.56it/s]


...

Epoch [17], Train Loss : [1.24183], Train Accuracy : [0.55419]


100%|██████████| 391/391 [00:14<00:00, 26.55it/s]


Epoch [18], Train Loss : [1.21635], Train Accuracy : [0.56495]


100%|██████████| 391/391 [00:14<00:00, 26.64it/s]

Epoch [19], Train Loss : [1.18232], Train Accuracy : [0.57617]


In [12]:
inference(my_resnet50, test_loader, device)

100%|██████████| 100/100 [00:02<00:00, 44.51it/s]

Test Accuracy: [0.59160]


In [14]:
torchvision_resnet18 = resnet18()
torchvision_resnet18.fc = nn.Linear(512, 10)

optimizer_resnet18 = optim.SGD(torchvision_resnet18.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0001)
train(torchvision_resnet18, train_loader, optimizer_resnet18, device)
inference(torchvision_resnet18, test_loader, device)

100%|██████████| 391/391 [00:06<00:00, 58.21it/s]


Epoch [0], Train Loss : [2.23649], Train Accuracy : [0.25477]


100%|██████████| 391/391 [00:06<00:00, 57.81it/s]


Epoch [1], Train Loss : [1.60244], Train Accuracy : [0.40620]


100%|██████████| 391/391 [00:06<00:00, 58.48it/s]


...

Epoch [18], Train Loss : [0.61419], Train Accuracy : [0.78728]


100%|██████████| 391/391 [00:06<00:00, 57.37it/s]


Epoch [19], Train Loss : [0.59584], Train Accuracy : [0.79270]


100%|██████████| 100/100 [00:01<00:00, 71.67it/s]

Test Accuracy: [0.76120]


In [16]:
torchvision_resnet50 = resnet50()
torchvision_resnet50.fc = nn.Linear(2048, 10)

optimizer_resnet50 = optim.SGD(torchvision_resnet50.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0001)
train(torchvision_resnet50, train_loader, optimizer_resnet50, device)
inference(torchvision_resnet50, test_loader, device)

100%|██████████| 391/391 [00:16<00:00, 23.60it/s]


Epoch [0], Train Loss : [4.60153], Train Accuracy : [0.16046]


100%|██████████| 391/391 [00:14<00:00, 27.53it/s]


Epoch [1], Train Loss : [2.04676], Train Accuracy : [0.25727]


100%|██████████| 391/391 [00:14<00:00, 27.61it/s]


...

Epoch [18], Train Loss : [1.15636], Train Accuracy : [0.58614]


100%|██████████| 391/391 [00:14<00:00, 27.59it/s]


Epoch [19], Train Loss : [1.13224], Train Accuracy : [0.59850]


100%|██████████| 100/100 [00:01<00:00, 55.05it/s]

Test Accuracy: [0.61070]
